Kütüphaneleri yükledik


In [4]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer  # for missing data
from sklearn import preprocessing  # for Dummy Variable
from sklearn.model_selection import train_test_split  # to split data into train and test
from sklearn.preprocessing import StandardScaler  # for scaling

Dataset'leri içe aktardık


In [3]:
train = pd.read_csv(
    'C:\\Users\\Genos\\Projeler\\Python İle Makine Öğrenmesi\\Projeler\\Titanic(ML)\\datasets\\train.csv')
test = pd.read_csv('C:\\Users\\Genos\\Projeler\\Python İle Makine Öğrenmesi\\Projeler\\Titanic(ML)\\datasets\\test.csv')
test_survived = pd.read_csv(
    'C:\\Users\\Genos\\Projeler\\Python İle Makine Öğrenmesi\\Projeler\\Titanic(ML)\\datasets\\gender_submission.csv')

NameError: name 'pd' is not defined

Burada sonrasında train test split veya cross validation kullanacağımız için hepsini tek bir data olarak topluyoruz ya da görselleştirmede daha büyük bir veri setimiz olduğu için farklı şeyler fark edebiliriz.

In [2]:
all_test = pd.merge(test, test_survived, on='PassengerId')
all_data = pd.concat([train, all_test], axis=0)

NameError: name 'pd' is not defined

In [5]:
all_data

NameError: name 'all_data' is not defined

In [3]:
# Age, Fare
imputer_num = SimpleImputer(missing_values=np.nan, strategy='mean')
all_data[['Age', 'Fare']] = imputer_num.fit_transform(all_data[['Age', 'Fare']])

imputer_cab = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='Unknown')
all_data['Cabin'] = pd.DataFrame(imputer_cab.fit_transform(all_data['Cabin'].values.reshape(1309, 1)))

imputer_emb = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
all_data['Embarked'] = pd.DataFrame(imputer_emb.fit_transform(all_data['Embarked'].values.reshape(1309, 1)))

In [ ]:
all_data.info()

In [4]:
# edited the index
all_data = all_data.iloc[:, 1:]
all_data.reset_index(inplace=True)
all_data.drop(['index'], axis=1, inplace=True)

In [5]:
# Sex, Embarked, Cabin, Ticket
oe = preprocessing.OrdinalEncoder()

all_data['Sex'] = oe.fit_transform(all_data['Sex'].values.reshape(-1, 1))
all_data['Embarked'] = oe.fit_transform(all_data['Embarked'].values.reshape(-1, 1))
all_data['Cabin'] = oe.fit_transform(all_data['Cabin'].values.reshape(-1, 1))
all_data['Ticket'] = oe.fit_transform(all_data['Ticket'].values.reshape(-1, 1))

In [6]:
# Pclass, Embarked
ohe = preprocessing.OneHotEncoder(sparse=False)

pclass = pd.DataFrame(ohe.fit_transform(all_data['Pclass'].values.reshape(-1, 1)))
embarked = pd.DataFrame(ohe.fit_transform(all_data['Embarked'].values.reshape(-1, 1)))

In [7]:
embarked_df = pd.DataFrame(data=embarked.values, index=range(len(embarked)), columns=['C', 'Q', 'S'])
pclass_df = pd.DataFrame(data=pclass.values, index=range(len(pclass)), columns=['1st', '2nd', '3rd'])

# drop the Pclass, Embarked, Name
all_data = all_data.drop(['Pclass', 'Embarked'], axis=1)

result0 = pd.concat([embarked_df, pclass_df], axis=1)
result = pd.concat([result0, all_data], axis=1)

result.drop('Name', axis=1, inplace=True)
x = result.drop('Survived', axis=1)
y = result['Survived']

In [8]:
# splitting data for training and testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=0)

In [39]:
result.index

RangeIndex(start=0, stop=1309, step=1)

In [9]:
# Scaling
sc = StandardScaler()

X_train = sc.fit_transform(x_train)
X_test = sc.fit_transform(x_test)

In [10]:
from sklearn.linear_model import LogisticRegression  # Linear Regression
from xgboost import XGBClassifier # Xgboost
from sklearn.metrics import confusion_matrix  # confusion matrix
import matplotlib.pyplot as plt
from sklearn.svm import SVC # SVC
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.naive_bayes import GaussianNB # Naive Bayes
from sklearn.tree import DecisionTreeClassifier # Decision Tree
from sklearn.ensemble import RandomForestClassifier # Random Forest
from sklearn.model_selection import cross_val_score # cross validation score

In [11]:
print('Logistic Regression', end='')
log_reg = LogisticRegression(random_state=0)
cv = cross_val_score(estimator=log_reg, X=X_train, y=y_train, cv=5)

print(cv)
print(cv.mean())

Logistic Regression[0.86931818 0.81818182 0.85714286 0.82285714 0.87428571]
0.8483571428571428


In [12]:
print('K-NN', end='')
knn = KNeighborsClassifier(n_neighbors=3, metric='minkowski')
cv = cross_val_score(estimator=knn, X=X_train, y=y_train, cv=5)

print(cv)
print(cv.mean())

K-NN[0.81818182 0.8125     0.81714286 0.82857143 0.8       ]
0.8152792207792208


In [22]:
print('SVC - kernel=rbf', end='')
svc = SVC(kernel='rbf', probability=True)
cv = cross_val_score(estimator=svc, X=X_train, y=y_train, cv=5)

print(cv)
print(cv.mean())

SVC - kernel=rbf[0.88068182 0.85227273 0.84571429 0.85142857 0.87428571]
0.8608766233766234


In [14]:
print('Navie Bayes - Gassian', end='')
gnb = GaussianNB()
cv = cross_val_score(estimator=gnb, X=X_train, y=y_train, cv=5)

print(cv)
print(cv.mean())

Navie Bayes - Gassian[0.8125     0.81818182 0.78857143 0.81142857 0.80571429]
0.8072792207792208


In [15]:
print('Decision Tree', end='')
dtc = DecisionTreeClassifier(criterion='entropy')
cv = cross_val_score(estimator=dtc, X=X_train, y=y_train, cv=5)

print(cv)
print(cv.mean())

Decision Tree[0.82386364 0.77840909 0.81714286 0.8        0.78857143]
0.8015974025974024


In [16]:
print('Random Forest', end='')
rfc = RandomForestClassifier(n_estimators=10, criterion='entropy')
cv = cross_val_score(estimator=rfc, X=X_train, y=y_train, cv=5)

print(cv)
print(cv.mean())

Random Forest[0.84659091 0.80113636 0.82857143 0.81714286 0.83428571]
0.8255454545454546


In [26]:
from sklearn.ensemble import VotingClassifier

classifiers = [('Logistic Regression', log_reg),
               ('KNN', knn),
               ('SVC', svc),
               ('Navie Bayes - Gassian', gnb),
               ('Decision Tree', dtc),
               ('Random Forest', rfc)]

voting_clf = VotingClassifier(estimators=classifiers, voting='hard')
cv = cross_val_score(estimator=voting_clf, X=X_train, y=y_train, cv=4)

print(cv)
print(cv.mean())

[0.85454545 0.84931507 0.85844749 0.83561644]
0.8494811124948112


In [45]:
x_test.sort_index()

,C,Q,S,1st,2nd,3rd,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,38.000000,1,0,816.0,71.2833,81.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,26.000000,0,0,914.0,7.9250,147.0
5,0.0,1.0,0.0,0.0,0.0,1.0,1.0,29.881138,0,0,373.0,8.4583,147.0
8,0.0,0.0,1.0,0.0,0.0,1.0,0.0,27.000000,0,2,477.0,11.1333,147.0
9,1.0,0.0,0.0,0.0,1.0,0.0,0.0,14.000000,1,0,174.0,30.0708,147.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,0.0,0.0,1.0,0.0,0.0,1.0,0.0,29.881138,0,0,394.0,7.7208,147.0
1302,0.0,1.0,0.0,1.0,0.0,0.0,0.0,37.000000,1,0,120.0,90.0000,147.0
1305,0.0,0.0,1.0,1.0,0.0,0.0,0.0,39.000000,0,0,834.0,108.9000,147.0
1306,0.0,0.0,1.0,0.0,0.0,1.0,1.0,38.500000,0,0,872.0,7.2500,147.0


In [44]:
voting_clf.fit(X_train, y_train)
y_hat_base_vc = voting_clf.predict(X_test).astype(int)

basic_submission = {'PassengerId': x_test.index, 'Survived': y_hat_base_vc}
base_submission = pd.DataFrame(data=basic_submission)

base_submission.to_csv('base_submission.csv', index=False)

[1 1 0 0 1 0 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 0 0
 1 1 0 0 0 1 0 0 0 1 1 1 1 1 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 1 0 1 0 1 1 1 0 1 0 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 1 1
 0 0 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 1 1 0 0
 0 0 0 1 1 0 1 0 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1 0 1 1 1 0 0 1 1 0 1 0 1 1
 1 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 1 0 1 0 1 0 0 0 0 0
 1 1 0 1 0 0 1 1 0 0 1 1 0 1 1 1 1 0 1 0 0 0 0 0 1 1 1 0 1 0 1 0 1 0 1 0 0
 1 1 0 1 1 0 1 0 1 0 0 1 1 0 0 1 0 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1 1 0 1 1 1
 1 0 1 1 1 1 1 0 0 0 0 0 1 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0
 0 1 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 0 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 1 0
 0 0 1 0 0 0 1 1 0 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0]
